In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import pandas as pd
import os
import re
from sqlalchemy import create_engine


### The Punch Newspaper

In [2]:
def punch_news():
    options = Options()
    options.headless = True
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')

    c = DesiredCapabilities.CHROME
    c["pageLoadStrategy"] = "none"
    #set chromodriver.exe path
    driver = webdriver.Chrome(r'C:\Users\HP\Downloads\news\News_station_analysis\chromedriver.exe',desired_capabilities=c,options=options)
    #explicit wait
    w = WebDriverWait(driver, 20)
    #launch URL
    driver.get("https://punchng.com/topics/news/")
    driver.implicitly_wait(20)
    time.sleep(3)
    #driver.implicitly_wait(20)
    #expected condition
    w.until(EC.presence_of_element_located((By.CLASS_NAME, 'post-title')))
    #JavaScript Executor to stop page load

    driver.execute_script("window.stop();")
    print('First huddle')

    content = []
    contents =driver.find_elements_by_class_name("post-title")
    for con in contents:
        cont = con.get_attribute('innerHTML')
        content.append(cont)
    af = pd.DataFrame(content,columns =['content'])
    af.content = af.content.apply(lambda x: x.replace('<a href=', ''))
    af.content = af.content.apply(lambda x: x.replace('</a>', ''))
    af.content = af.content.apply(lambda x: x.replace('>', '|'))
    af = af['content'].str.split("|",n = 3, expand = True)
    af.columns = ['link','title']
    af = af.drop_duplicates(subset=["link"], keep='first')
    print('Second huddle')
    full_contents = []
    dates = []
    by = []
    def all_news(ev):
        h = WebDriverWait(driver, 20)
        full = []
        timed = []
        print('Pages extraction in progress')

        driver.get(ev)
        time.sleep(4)
        driver.implicitly_wait(20)
        h.until(EC.presence_of_element_located((By.CLASS_NAME, 'post-content')))
        #JavaScript Executor to stop page load
        driver.execute_script("window.stop();")
        full_content = driver.find_elements_by_class_name("post-content")
        for conten in full_content:
            co = conten.get_attribute('innerText')
            co1 = co.replace('\n\n',' ')
            co2 = co1.replace('\n',' ')
            co3 = co2.split(',', 1)
            full.append(co3)

        date = driver.find_elements_by_class_name("col-lg-4")
        for dat in date:
            dat1= dat.get_attribute('innerText')
            dat2 = dat1.replace('By\xa0\n','')
            timed.append(dat2)
        full_contents.append(full[0])
        dates.append(timed[0])
        by.append(timed[1])

    m =af.link.to_list()
    m =  [item.replace('"', '') for item in m]
    for o in m:
        all_news(o)

    driver.quit()

    aa = pd.DataFrame({'Title':af.title,'Full_content': full_contents,'Date':dates,'Author':by,'Source_link':af.link})
    ff  =aa['Full_content'].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))#unlist the full_content column

    aa['Words_count'] = ff.str.split().str.len()#counts the full_content
    n = open("negative-words.txt", "r")
    p = open("positive-words.txt", "r")
    n_word = n.read()
    p_word = p.read()
    n.close()
    p.close()
    n_word=n_word.replace('\n',',')
    n_word = re.sub("[^\w]", " ", n_word).split()
    p_word=p_word.replace('\n',',')
    p_word = re.sub("[^\w]", " ", p_word).split()
    aa['Full_content'] = ff
    #df['word_overlap'] = [set(x[0].split()) & set(x[1].split()) for x in df.values]
    def negative_words(x):
        negative_score = 0
        for word in n_word:
            if word in x:
                negative_score += 1
        return negative_score

    def positive_words(x):
        positive_score = 0
        for word in p_word:
            if word in x:
                positive_score += 1
        return positive_score
    aa['Negative_words'] = aa['Full_content'].apply(lambda x : negative_words(x))
    aa['Positive_words'] = aa['Full_content'].apply(lambda x : positive_words(x))
    aa['Sentence_count'] = aa['Full_content'].str.count('[\w][\.!\?]')
    aa['Sentiment'] = round((aa['Positive_words'] - aa['Negative_words']) / aa['Words_count'], 2)
    aa['News_type'] = ['Bad News' if x < 0 else 'Good News' if x > 0 else 'Neutral' for x in aa.Sentiment]
    engine = create_engine('sqlite:///news.db')
    aa.to_sql('punch_data', engine, if_exists='append', index=False)
    

In [3]:
punch_news()

First huddle
Second huddle
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress


## Vanguard Newspaper

In [8]:
def vanguard_news():
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    options.headless = True
    c = DesiredCapabilities.CHROME
    c["pageLoadStrategy"] = "none"
    #set chromodriver.exe path

    driver = webdriver.Chrome(r'C:\Users\HP\Downloads\news\News_station_analysis\chromedriver.exe',desired_capabilities=c,options=options)
    #explicit wait

    driver.get("https://www.vanguardngr.com/news/")
    #explicit wait
    w = WebDriverWait(driver, 20)
    #launch URL
    #driver.get("https://www.vanguardngr.com/category/headlines/")
    driver.implicitly_wait(20)
    time.sleep(3)
    #driver.implicitly_wait(20)
    #expected condition
    w.until(EC.presence_of_element_located((By.CLASS_NAME, 'entry-title')))
    #JavaScript Executor to stop page load

    driver.execute_script("window.stop();")
    print('First huddle')

    content = []
    contents =driver.find_elements_by_class_name("entry-title")
    for con in contents:
        cont = con.get_attribute('innerHTML')
        content.append(cont)

    af = pd.DataFrame(content,columns =['content'])
    af.content = af.content.apply(lambda x: x.replace('<a href="', ''))
    af = af.iloc[1:]
    af = af.reset_index()
    af.content = af.content.apply(lambda x: x.replace('rel="bookmark">', ' | '))
    af.content = af.content.apply(lambda x: x.replace('</a>', ' '))
    af = af['content'].str.split("|",n = 3, expand = True)
    af.columns = ['News_link','Title']
    sd = af.head(20)
    driver.quit()
    full_contents = []
    dates = []
    datetime = []
    genres = []
    def all_news(ev):
        options = Options()
        options.headless = True
        ua = UserAgent()
        userAgent = ua.random
        options.page_load_strategy = 'eager'
        options.add_argument(f'user-agent={userAgent}')
        driver = webdriver.Chrome(r'C:\Users\HP\Downloads\news\News_station_analysis\chromedriver.exe',options=options)
        driver.get(ev)
        driver.implicitly_wait(20)
        time.sleep(10)

        full_content = driver.find_elements_by_class_name("entry-content")
        for conten in full_content:
            co = conten.get_attribute('innerText')
            co1 = co.replace('\n\n','')
            co2 = co1.replace('Subscribe for latest Videos','')
            #co3 = co2[co2.find('|'):] #deletes anything before the |
            co3 = co2.replace('\n',' ')
            co4 = co3.split(',',1)
            full_contents.append(co4)

        date = driver.find_elements_by_class_name("entry-date.published.updated")
        for dat in date:
            date= dat.get_attribute('innerText')
            dates.append(date)
            tim= dat.get_attribute('dateTime')
            datetime.append(tim)

        genre = driver.find_elements_by_xpath("""//*[@id="main"]/header/span/a""")
        for gen in genre:
            gen1= gen.get_attribute('innerText')
            #gen2 = gen1.replace('POSTED IN\n','')
            genres.append(gen1)
        print('going')
        driver.quit()

    a = sd.News_link.to_list()
    a =  [item.replace('"  ', '') for item in a]
    #a = ['https://www.vanguardngr.com/2022/12/mavins-marks-10th-anniversary-with-new-album/']
    for i in a:
        all_news(i)

    ss = pd.DataFrame({'Full_content': full_contents,'Date':dates,'Time_published':datetime})
    ff  =ss['Full_content'].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))

    ss['Words_count'] = ff.str.split().str.len()
    n = open("negative-words.txt", "r")
    p = open("positive-words.txt", "r")
    n_word = n.read()
    p_word = p.read()
    n.close()
    p.close()
    n_word=n_word.replace('\n',',')
    n_word = re.sub("[^\w]", " ", n_word).split()
    p_word=p_word.replace('\n',',')
    p_word = re.sub("[^\w]", " ", p_word).split()
    #df['word_overlap'] = [set(x[0].split()) & set(x[1].split()) for x in df.values]
    def negative_words(x):
        negative_score = 0
        for word in n_word:
            if word in x:
                negative_score += 1
        return negative_score

    def positive_words(x):
        positive_score = 0
        for word in p_word:
            if word in x:
                positive_score += 1
        return positive_score
    ss['Full_content'] = ff
    ss['News_genre'] = genres
    ss['Negative_words'] = ss['Full_content'].apply(lambda x : negative_words(x))
    ss['Positive_words'] = ss['Full_content'].apply(lambda x : positive_words(x))
    ss['Sentence_count'] = ss['Full_content'].str.count('[\w][\.!\?]')

    ss['Sentiment'] = round((ss['Positive_words'] - ss['Negative_words']) / ss['Words_count'], 2)
    ss['News_type'] = ['Bad News' if x < 0 else 'Good News' if x > 0 else 'Neutral' for x in ss.Sentiment]
    va = pd.concat([sd,ss], axis=1)
    engine = create_engine('sqlite:///news.db')
    va.to_sql('vanguard_data', engine, if_exists='append', index=False)

In [9]:
vanguard_news()


First huddle
going
going
going
going
going
going
going
going
going
going
going
going
going
going
going
going
going
going
going
going


## The Nation Newspaper

In [95]:
options = Options()
#options.headless = True
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')

c = DesiredCapabilities.CHROME
c["pageLoadStrategy"] = "none"
#set chromodriver.exe path
driver = webdriver.Chrome(r'C:\Users\HP\Downloads\news\News_station_analysis\chromedriver.exe',desired_capabilities=c,options=options)
#explicit wait
w = WebDriverWait(driver, 20)
#launch URL
driver.get("https://thenationonlineng.net/news/")
driver.implicitly_wait(20)

#driver.implicitly_wait(20)
#expected condition
w.until(EC.presence_of_element_located((By.CLASS_NAME, 'nation_category_post_title')))
#JavaScript Executor to stop page load

driver.execute_script("window.stop();")
print('First huddle')

content = []
contents =driver.find_elements_by_css_selector('section section a')
for con in contents:
    cont = con.get_attribute('href')
    content.append(cont)

af = pd.DataFrame(content,columns =['content'])
af = af.drop_duplicates(subset=["content"], keep='first')
#af = af[af["content"].str.contains("https://thenationonlineng.net/news/page/2/") == False]
discard = ["https://thenationonlineng.net/news/page/"]
  
# drop rows that contain the partial string "Sci"
af = af[~af.content.str.contains('|'.join(discard))]
titless = []
by = []
date = []
full_content = []

def all_news(ev):
    h = WebDriverWait(driver, 20)
    driver.get(ev)
    time.sleep(5)
    driver.implicitly_wait(20)
    h.until(EC.presence_of_element_located((By.CLASS_NAME, 'nation__article__content')))
    #JavaScript Executor to stop page load
    driver.execute_script("window.stop();")
    title = driver.find_elements_by_css_selector('section article header h1')
    for tit in title:
        titl = tit.get_attribute('innerText')
        titless.append(titl)

    who = driver.find_elements_by_class_name("nation__article__meta")
    for wh in who:
        w = wh.get_attribute('outerText')
        w = w.replace('\n',' | ')
        w = w.split('|',1)

        by.append(w[0])
        date.append(w[1])
    print('Pages extraction in progress')
    article = driver.find_elements_by_class_name('nation__article__content')
    for art in article:
        arti = art.get_attribute('innerText')
        arti = arti.replace('\n\n','')
        arti = arti.split('ADVERTISEMENTS')[0]
        full_content.append(arti)
        
for i in af.content:
    all_news(i)
driver.quit()  
ss = pd.DataFrame({'Title': titless,'Full_content': full_content,'Date':date,'Author':by,'News_link':af.content})
ss['Words_count'] = ss.Full_content.str.split().str.len()


First huddle
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress
Pages extraction in progress


,Title,Full_content,Date,Author,News_link
0,"Soludo, Obi, Obiano reunite as Cardinal Arinze...",It was a kind of reunion between Anambra Gover...,"December 8, 2022","Nwanosike Onu, Awka and Emma Elekwa, Onitsha",https://thenationonlineng.net/soludo-obi-obian...
1,"Rivers, Atiku’s campaigner bicker over gunmen ...",The Rivers State Government and chairman of th...,"December 8, 2022","Mike Odiegwu, Port Harcourt",https://thenationonlineng.net/rivers-atikus-ca...
2,‘Buhari will leave Nigeria better than he met it’,"The Chief of Staff to the President, Prof. Ibr...","December 8, 2022",Agency Reporter,https://thenationonlineng.net/buhari-will-leav...
3,No going back on cash withdrawal limit policy ...,The Central Bank of Nigeria (CBN) has stated t...,"December 8, 2022","Bolaji Ogundele, Abuja",https://thenationonlineng.net/no-going-back-on...
4,Reps summon Emefiele over cash withdrawal policy,The House of Representatives on Thursday summo...,"December 8, 2022",Agency Reporter,https://thenationonlineng.net/updated-reps-sum...
5,"953,803 PVCs uncollected in Lagos, says INEC",The Independent National Electoral Commission ...,"December 8, 2022",Agency Reporter,https://thenationonlineng.net/953803-pvcs-unco...
6,"I won’t be distracted, says Wike",Rivers Governor Nyesom Wike has vowed that not...,"December 8, 2022","Mike Odiegwu, Port Harcourt",https://thenationonlineng.net/i-wont-be-distra...
7,Court dismisses suit challenging Adeola’s Ogun...,"AFederal High Court sitting in Abeokuta, the O...","December 8, 2022","Ernest Nwokolo, Abeokuta",https://thenationonlineng.net/court-dismisses-...
8,Gunmen whisk away four new babies in Soludo’s ...,Gunmen in the early hours of Thursday allegedl...,"December 8, 2022","Nwanosike Onu, Awka",https://thenationonlineng.net/gunmen-whisk-awa...
9,Supreme Court dismisses appeal against PDP’s I...,The Supreme Court has dismissed an appeal agai...,"December 8, 2022","Eric Ikhilae, Abuja",https://thenationonlineng.net/supreme-court-di...


In [100]:
n = open("negative-words.txt", "r")
p = open("positive-words.txt", "r")
n_word = n.read()
p_word = p.read()
n.close()
p.close()
n_word=n_word.replace('\n',',')
n_word = re.sub("[^\w]", " ", n_word).split()
p_word=p_word.replace('\n',',')
p_word = re.sub("[^\w]", " ", p_word).split()
def negative_words(x):
    negative_score = 0
    for word in n_word:
        if word in x:
            negative_score += 1
    return negative_score

def positive_words(x):
    positive_score = 0
    for word in p_word:
        if word in x:
            positive_score += 1
    return positive_score

ss['Negative_words'] = ss['Full_content'].apply(lambda x : negative_words(x))
ss['Positive_words'] = ss['Full_content'].apply(lambda x : positive_words(x))
ss['Sentence_count'] = ss['Full_content'].str.count('[\w][\.!\?]')

ss['Sentiment'] = round((ss['Positive_words'] - ss['Negative_words']) / ss['Words_count'], 2)
ss['News_type'] = ['Bad News' if x < 0 else 'Good News' if x > 0 else 'Neutral' for x in ss.Sentiment]
engine = create_engine('sqlite:///news.db')
ss.to_sql('nation_newspaper_data', engine, if_exists='append', index=False)